In [32]:
from bs4 import BeautifulSoup
import urllib
import re

In [125]:
search_url = "http://provincialsearch.thehealthline.ca/search?q=job&btnG=GO&site=torontocentralhealthline&getfields=*&client=torontocentralhealthline&proxystylesheet=torontocentralhealthline&output=xml_no_dtd&oe=UTF-8&ie=UTF-8&filter=0&btnG.x=36&btnG.y=5&numgm=4&entsp=a&wc=200&wc_mc=1&ud=1&tlen=150@start%3D10&ulang=en&ip=205.189.187.4&access=p&sort=date:D:L:d1&entqr=3&entqrm=0"

In [126]:
service_links = []
for i in range(0,240,10):
    print "getting page " + str(i / 10) 
    url = search_url + "&start=" + str(i)
    print url
    search_html = urllib.urlopen(url).read()
    soup = BeautifulSoup(search_html)
    for j in soup.find("div",{"class":"main-results-without-dn"}).findAll("a"):
        service_link =  j['href']
        print service_link
        if "torontocentralhealthline.ca" in service_link:
            service_links.append(service_link)

getting page 0
http://provincialsearch.thehealthline.ca/search?q=job&btnG=GO&site=torontocentralhealthline&getfields=*&client=torontocentralhealthline&proxystylesheet=torontocentralhealthline&output=xml_no_dtd&oe=UTF-8&ie=UTF-8&filter=0&btnG.x=36&btnG.y=5&numgm=4&entsp=a&wc=200&wc_mc=1&ud=1&tlen=150@start%3D10&ulang=en&ip=205.189.187.4&access=p&sort=date:D:L:d1&entqr=3&entqrm=0&start=0
http://www.torontocentralhealthline.ca/listServices.aspx?id=10715
http://www.torontocentralhealthline.ca/displayservice.aspx?id=132705
http://www.torontocentralhealthline.ca/displayservice.aspx?id=144083
http://www.torontocentralhealthline.ca/displayservice.aspx?id=132628
http://www.torontocentralhealthline.ca/displayservice.aspx?id=132700
http://www.torontocentralhealthline.ca/displayservice.aspx?id=169551
http://www.lignesantecentre-toronto.ca/displayservice.aspx?id=169551
http://www.torontocentralhealthline.ca/displayservice.aspx?id=133601
http://www.torontocentralhealthline.ca/displayservice.aspx?id=

In [45]:
#service_links

In [127]:
import pandas as pd

In [128]:
#service structure
import collections
section = collections.namedtuple('section', 'element id header')

program = section("span", "ctl00_ContentPlaceHolder1_lblProgram", "program")
organization = section("a", "ctl00_ContentPlaceHolder1_rptOrganizations_ctl00_lnkOrganization", "organization")
phone_number = section("span", "ctl00_ContentPlaceHolder1_lblOfficePhone", "phone_number")
fax_number = section("span", "ctl00_ContentPlaceHolder1_lblFax", "fax_number")
email = section("a", "ctl00_ContentPlaceHolder1_lnkEmail", "email")
website = section("a", "ctl00_ContentPlaceHolder1_lnkUrl", "website")
address = section("span", "ctl00_ContentPlaceHolder1_lblAddress", "address")
intersection = section("span", "ctl00_ContentPlaceHolder1_lblIntersection", "intersection")
hours = section("span", "ctl00_ContentPlaceHolder1_lblHours", "hours")
contact = section("span", "ctl00_ContentPlaceHolder1_lblContact", "contact")
description = section("span", "ctl00_ContentPlaceHolder1_lblDescription", "description")
fees = section("span", "ctl00_ContentPlaceHolder1_lblFees", "fees")
application = section("span", "ctl00_ContentPlaceHolder1_lblApplication", "application")
eligibility = section("span", "ctl00_ContentPlaceHolder1_lblEligibility", "eligibility")
languages = section("span", "ctl00_ContentPlaceHolder1_lblLanguages", "languages")
areas_served = section("span","ctl00_ContentPlaceHolder1_lblAreasServed", "areas_served")

sections = [program,organization,phone_number,fax_number,email,
            website,address,hours,contact,description,fees,application,
            eligibility,languages,areas_served]

In [130]:
df = pd.DataFrame()
id_pattern = re.compile(r".*id=([0-9]+).*")

counter = 0
total = len(service_links)
for service_link in service_links:
    try:
        print("Grabbing service {0} of {1}".format(counter,total))
        service_page = urllib.urlopen(service_link).read()
        soup_service = BeautifulSoup(service_page)
        service_id = re.match(id_pattern,service_link).groups()[0]
        res = ""
        for section in sections:
            res = soup_service.find(section.element, {"id":section.id})
            try:
                df.loc[service_id,section.header] =  res.contents[0]
            except:
                try:
                    df.loc[service_id,section.header] = res.text
                except:
                    df.loc[service_id,section.header] = None
        counter += 1
    except:
        counter += 1
        pass

Grabbing service 0 of 262
Grabbing service 1 of 262
Grabbing service 2 of 262
Grabbing service 3 of 262
Grabbing service 4 of 262
Grabbing service 5 of 262
Grabbing service 6 of 262
Grabbing service 7 of 262
Grabbing service 8 of 262
Grabbing service 9 of 262
Grabbing service 10 of 262
Grabbing service 11 of 262
Grabbing service 12 of 262
Grabbing service 13 of 262
Grabbing service 14 of 262
Grabbing service 15 of 262
Grabbing service 16 of 262
Grabbing service 17 of 262
Grabbing service 18 of 262
Grabbing service 19 of 262
Grabbing service 20 of 262
Grabbing service 21 of 262
Grabbing service 22 of 262
Grabbing service 23 of 262
Grabbing service 24 of 262
Grabbing service 25 of 262
Grabbing service 26 of 262
Grabbing service 27 of 262
Grabbing service 28 of 262
Grabbing service 29 of 262
Grabbing service 30 of 262
Grabbing service 31 of 262
Grabbing service 32 of 262
Grabbing service 33 of 262
Grabbing service 34 of 262
Grabbing service 35 of 262
Grabbing service 36 of 262
Grabbing se

In [131]:
df.to_csv("job_services.csv",encoding="UTF-8",index=False)